**Installing OpenAI Package**
To use the OpenAI API, we need to install the `openai` package. We can do this using the `pip` command.

In [ ]:
import subprocess
import sys

def install_openai():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "openai"])

install_openai()

**OpenAI API & Functions**
Pizza ordeing chat demo.
Ex. Add another function and execue it during the chat.

In [ ]:
import os
import openai
import json
import sys

key = "OPEN-AI-KEY"

def place_order(name, size, take_away=False):
    place = "take away" if take_away else "eat in"
    print("\033[31mplace_order function is executed: \033[0m", end="")
    print(f"Placing order for {name} pizza, {size} to {place}")


def call_staff():
    print("\033[31mcall_staff function is executed: \033[0m", end="")
    print("Calling staff to table")


def get_chat(
    openai_client, messages=None, model="gpt-4", temperature=0.2, functions=None #3.5 will not work ok
):
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        functions=functions,
    )
    message = response.choices[0].message
    if message.function_call:
        f = globals()[message.function_call.name]
        params = json.loads(message.function_call.arguments)
        f(**params)
    else:
        print("\033[94mPizza helper:\033[0m", message.content)


    return message.function_call is not None


def main():
    functions = [
        {
            "name": "place_order",
            "description": "Place an order for a pizza",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The name of the pizza, e.g. Pepperoni",
                    },
                    "size": {
                        "type": "string",
                        "enum": ["small", "medium", "large"],
                        "description": "The size of the pizza. Always ask for clarification if not specified.",
                    },
                    "take_away": {
                        "type": "boolean",
                        "description": "Whether the pizza is taken away. Assume false if not specified.",
                    },
                },
                "required": ["name", "size", "take_away"],
            },
        },
        {
            "name": "call_staff",
            "description": "Call a member of staff to the table",
            "parameters": {"type": "object", "properties": {}},
        },
    ]

    messages = [
        {
            "role": "system",
            "content": "Don't make assumptions about what values to put into functions. "
            + "Ask for clarification if you need to.",
        }
    ]

    function_called = False
    openai_client = openai.OpenAI(
        api_key=key)

    print("\033[94mPizza helper:\033[0m", "How may I help you? Would you like a pizza?")

    try:
        while not function_called:
            user_input = input("You: ")
            messages.append({"role": "user", "content": user_input})
            function_called = get_chat(
                openai_client, messages=messages, functions=functions
            )
    except KeyboardInterrupt:
        print("\n\033[94mPizza helper:\033[0m", "Thank you. Bye Bye")


if __name__ == "__main__":
    main()